In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

def calculate_displacement(ref_frame, target_frame):
    # Convert frames to grayscale
    ref_gray = cv2.cvtColor(ref_frame, cv2.COLOR_BGR2GRAY)
    target_gray = cv2.cvtColor(target_frame, cv2.COLOR_BGR2GRAY)

    # Calculate the absolute difference (retain all minor changes)
    displacement = cv2.absdiff(ref_gray, target_gray)

    # No thresholding and no blurring
    return displacement


def generate_heatmap(displacement_array):
    # Dynamic normalization based on actual data
    norm = Normalize(vmin=np.min(displacement_array), vmax=np.max(displacement_array))
    colormap = plt.cm.magma_r

    heatmap = colormap(norm(displacement_array), bytes=False)
    heatmap_8bit = (heatmap * 255).astype(np.uint8)
    heatmap_bgr = cv2.cvtColor(heatmap_8bit, cv2.COLOR_RGBA2BGR)

    return heatmap_bgr


def stabilize_frame(ref_frame, current_frame):
    # Feature detection and matching
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(ref_frame, None)
    kp2, des2 = orb.detectAndCompute(current_frame, None)
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = matcher.match(des1, des2)

    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = kp1[match.queryIdx].pt
        points2[i, :] = kp2[match.trainIdx].pt

    # Find homography
    h, mask = cv2.findHomography(points2, points1, cv2.RANSAC)

    # Use homography to stabilize the frame
    height, width = ref_frame.shape[:2]
    stabilized_frame = cv2.warpPerspective(current_frame, h, (width, height))

    return stabilized_frame

def process_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)
    ret, ref_frame = cap.read()  # Assuming the first frame as the reference frame

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
    out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Stabilize the frame
        stabilized_frame = stabilize_frame(ref_frame, frame)

        # Calculate displacement on stabilized frame
        displacement = calculate_displacement(ref_frame, stabilized_frame)
        heatmap = generate_heatmap(displacement)
        out.write(heatmap)

    cap.release()
    out.release()

# Example usage
process_video('input1.mov', 'output_test2.mov')